In [ ]:
#library
import string
import re
import pandas as pd
import numpy as np
from google.colab import drive
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
!pip install swifter
import swifter 
!pip install indoNLP

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 30.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 821 kB 7.1 MB/s 
     |████████████████████████████████| 281 kB 40.9 MB/s 
     |████████████████████████████████| 141 kB 58.4 MB/s 
  Created wheel for swifter: filename=swifter-1.3.3-py3-none-any.whl size=16253 sha256=93eb0fcb190b65a1fef0e629689206e515119f45b4b7c8885d24de44f8450b52
  Stored in directory: /root/.cache/pip/wheels/64/bf/da/da0022edab5fd84114858a95e4f32f2fca0d5b7d758905f594
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 29.4 MB/s 


In [ ]:
#Menyambungkan Ke drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## Import data 
data_pakai = pd.read_excel('/content/gdrive/MyDrive/Skripsi/Data/data_pakai.xlsx')
data_pakai.head()

,service,date,time,message,sentimen
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral


In [ ]:
print(data_pakai.shape)

(4354, 5)


In [ ]:
data_pakai['sentimen'].value_counts()

Negatif    1630
Positif    1369
Netral     1355
Name: sentimen, dtype: int64

## Data Cleansing

In [ ]:
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, words_to_emoji

In [ ]:
def data_cleansing(text):
    ## Emoji 
    text = emoji_to_words(text)
    ## Hapus tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    ## Hapus non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    #  Hapus mention, link, hashtag
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    #Hapus number
    text = re.sub(r"\d+", "", text)
    #remove punctuation
    text = text.translate(str.maketrans("","",string.punctuation))
    #remove whitespace leading & trailing
    text = text.strip()
    #remove multiple whitespace into single whitespace
    text = re.sub('\s+',' ',text)
    # remove single char
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    # word elongatin
    text = replace_word_elongation(text)
    return text

In [ ]:
data_pakai['cleansing'] = data_pakai['message'].apply(data_cleansing)
data_pakai.head()

,service,date,time,message,sentimen,cleansing
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif,RT PPKM Pernah percaya kemudian menyesal
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif,RT Cegah Penyebaran Covid Kapolsek Selbar bagi...
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif,SekMaos Pemerintah merevisi aturan Pemberlakua...
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif,RT Info Rekaman CCTV Oknum Satpol P di Kab Gow...
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral,RT Kemenag mengumumkan petunjuk teknis pelaksa...


## Case Folding

In [ ]:
def case_folding(text):
  text = text.lower()
  return text 

In [ ]:
data_pakai['case folding'] = data_pakai['cleansing'].apply(case_folding)
data_pakai.head()

,service,date,time,message,sentimen,cleansing,case folding
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif,RT PPKM Pernah percaya kemudian menyesal,rt ppkm pernah percaya kemudian menyesal
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif,RT Cegah Penyebaran Covid Kapolsek Selbar bagi...,rt cegah penyebaran covid kapolsek selbar bagi...
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif,SekMaos Pemerintah merevisi aturan Pemberlakua...,sekmaos pemerintah merevisi aturan pemberlakua...
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif,RT Info Rekaman CCTV Oknum Satpol P di Kab Gow...,rt info rekaman cctv oknum satpol p di kab gow...
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral,RT Kemenag mengumumkan petunjuk teknis pelaksa...,rt kemenag mengumumkan petunjuk teknis pelaksa...


## Tokeneizing

In [ ]:
def word_tokenize_wrapper(text):
  text = word_tokenize(text)
  return text

In [ ]:
data_pakai['Tokeneizing'] = data_pakai['case folding'].apply(word_tokenize_wrapper)
data_pakai.head()

,service,date,time,message,sentimen,cleansing,case folding,Tokeneizing
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif,RT PPKM Pernah percaya kemudian menyesal,rt ppkm pernah percaya kemudian menyesal,"[rt, ppkm, pernah, percaya, kemudian, menyesal]"
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif,RT Cegah Penyebaran Covid Kapolsek Selbar bagi...,rt cegah penyebaran covid kapolsek selbar bagi...,"[rt, cegah, penyebaran, covid, kapolsek, selba..."
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif,SekMaos Pemerintah merevisi aturan Pemberlakua...,sekmaos pemerintah merevisi aturan pemberlakua...,"[sekmaos, pemerintah, merevisi, aturan, pember..."
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif,RT Info Rekaman CCTV Oknum Satpol P di Kab Gow...,rt info rekaman cctv oknum satpol p di kab gow...,"[rt, info, rekaman, cctv, oknum, satpol, p, di..."
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral,RT Kemenag mengumumkan petunjuk teknis pelaksa...,rt kemenag mengumumkan petunjuk teknis pelaksa...,"[rt, kemenag, mengumumkan, petunjuk, teknis, p..."


## Normalization

In [ ]:
normalizad_word = pd.read_csv("/content/gdrive/MyDrive/Skripsi/Data/colloquial-indonesian-lexicon.csv")
normalizad_word_dict = {}
for index, row in normalizad_word.iterrows():
  if row[0] not in normalizad_word_dict:
      normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
  return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

In [ ]:
data_pakai['normalized'] = data_pakai['Tokeneizing'].apply(normalized_term)
data_pakai[['Tokeneizing','normalized']].head(20)

,Tokeneizing,normalized
0,"[rt, ppkm, pernah, percaya, kemudian, menyesal]","[rt, ppkm, pernah, percaya, kemudian, menyesal]"
1,"[rt, cegah, penyebaran, covid, kapolsek, selba...","[rt, cegah, penyebaran, covid, kapolsek, selba..."
2,"[sekmaos, pemerintah, merevisi, aturan, pember...","[sekmaos, pemerintah, merevisi, aturan, pember..."
3,"[rt, info, rekaman, cctv, oknum, satpol, p, di...","[rt, info, rekaman, cctv, oknum, satpol, p, di..."
4,"[rt, kemenag, mengumumkan, petunjuk, teknis, p...","[rt, kemenag, mengumumkan, petunjuk, teknis, p..."
5,"[rt, kamu, mau, ga, jadi, pacar, aku, nanti, y...","[rt, kamu, mau, enggak, jadi, pacar, aku, nant..."
6,"[zahran, kasus, berkurang, dengki, meningkat]","[zahran, kasus, berkurang, dengki, meningkat]"
7,"[rt, tan, kemaren, psb, sekarang, ppkm, bentar...","[rt, tan, kemarin, psb, sekarang, ppkm, bentar..."
8,"[ners, maap, oot, aku, udah, beres, isoman, ud...","[ners, maaf, oot, aku, sudah, beres, isoman, s..."
9,"[rt, ppkm, diperpanjang, terus, kapan, kelarny...","[rt, ppkm, diperpanjang, terus, kapan, kelarny..."


## Steaming 

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_pakai['normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

Output streaming akan dipotong hingga 5000 baris terakhir.
dilema : dilema
penambahan : tambah
labil : labil
pungky : pungky
instruktur : instruktur
penyelenggara : selenggara
senam : senam
puri : puri
lantaran : lantar
kegiatannya : giat
menyesali : sali
perbuatannya : buat
piknik : piknik
krb : krb
terancam : ancam
dipenjara : penjara
manajer : manajer
rhnews : rhnews
pesantren : pesantren
bktm : bktm
banaran : banar
aiptu : aiptu
mashudi : mashudi
serda : serda
reiner : reiner
imbir : imbir
pendampingan : damping
monitoring : monitoring
jps : jps
kpm : kpm
amindonesia : amindonesia
tuiter : tuiter
du : du
yor : yor
mejik : mejik
wasit : wasit
cewek : cewek
felinda : felinda
dipanggil : panggil
fe : fe
bersangkutan : sangkut
indro : indro
nora : nora
ustad : ustad
somad : somad
lumajang : lumajang
polripresisi : polripresisi
divisihumaspolri : divisihumaspolri
humaspoldajatim : humaspoldajatim
polreslumajang : polreslumajang
hobby : hobby
telor : telor
manfaatnya : manfaat
taktis : t

In [ ]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

In [ ]:
data_pakai['steaming'] = data_pakai['normalized'].swifter.apply(get_stemmed_term)
data_pakai.head()

Pandas Apply:   0%|          | 0/4354 [00:00<?, ?it/s]

,service,date,time,message,sentimen,cleansing,case folding,Tokeneizing,normalized,steaming
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif,RT PPKM Pernah percaya kemudian menyesal,rt ppkm pernah percaya kemudian menyesal,"[rt, ppkm, pernah, percaya, kemudian, menyesal]","[rt, ppkm, pernah, percaya, kemudian, menyesal]","[rt, ppkm, pernah, percaya, kemudian, sesal]"
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif,RT Cegah Penyebaran Covid Kapolsek Selbar bagi...,rt cegah penyebaran covid kapolsek selbar bagi...,"[rt, cegah, penyebaran, covid, kapolsek, selba...","[rt, cegah, penyebaran, covid, kapolsek, selba...","[rt, cegah, sebar, covid, kapolsek, selbar, ba..."
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif,SekMaos Pemerintah merevisi aturan Pemberlakua...,sekmaos pemerintah merevisi aturan pemberlakua...,"[sekmaos, pemerintah, merevisi, aturan, pember...","[sekmaos, pemerintah, merevisi, aturan, pember...","[sekmaos, perintah, revisi, atur, laku, batas,..."
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif,RT Info Rekaman CCTV Oknum Satpol P di Kab Gow...,rt info rekaman cctv oknum satpol p di kab gow...,"[rt, info, rekaman, cctv, oknum, satpol, p, di...","[rt, info, rekaman, cctv, oknum, satpol, p, di...","[rt, info, rekam, cctv, oknum, satpol, p, di, ..."
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral,RT Kemenag mengumumkan petunjuk teknis pelaksa...,rt kemenag mengumumkan petunjuk teknis pelaksa...,"[rt, kemenag, mengumumkan, petunjuk, teknis, p...","[rt, kemenag, mengumumkan, petunjuk, teknis, p...","[rt, kemenag, umum, tunjuk, teknis, laksana, k..."


In [ ]:
#data_pakai.to_csv('data_baru+Stopword.csv', index = False)

## Filtering (Optional)

In [ ]:
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'd"' 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh',
                       '&amp', 'yah', 'wkwk'])

In [ ]:
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

In [ ]:
data_pakai['filtering'] = data_pakai['steaming'].apply(stopwords_removal)
data_pakai.head()

,service,date,time,message,sentimen,cleansing,case folding,Tokeneizing,normalized,steaming,filtering
0,twitter,07/04/2021,8:47:00,RT @fsskroeppreal: PPKM - Pernah percaya kemud...,Negatif,RT PPKM Pernah percaya kemudian menyesal,rt ppkm pernah percaya kemudian menyesal,"[rt, ppkm, pernah, percaya, kemudian, menyesal]","[rt, ppkm, pernah, percaya, kemudian, menyesal]","[rt, ppkm, pernah, percaya, kemudian, sesal]","[ppkm, percaya, sesal]"
1,twitter,10/16/2021,8:34:00,"RT @PolsekPupuan:Cegah Penyebaran Covid-19, Ka...",Positif,RT Cegah Penyebaran Covid Kapolsek Selbar bagi...,rt cegah penyebaran covid kapolsek selbar bagi...,"[rt, cegah, penyebaran, covid, kapolsek, selba...","[rt, cegah, penyebaran, covid, kapolsek, selba...","[rt, cegah, sebar, covid, kapolsek, selbar, ba...","[cegah, sebar, covid, kapolsek, selbar, masker..."
2,twitter,07/12/2021,11:56:00,@Humas_SekMaos Pemerintah merevisi aturan Pemb...,Positif,SekMaos Pemerintah merevisi aturan Pemberlakua...,sekmaos pemerintah merevisi aturan pemberlakua...,"[sekmaos, pemerintah, merevisi, aturan, pember...","[sekmaos, pemerintah, merevisi, aturan, pember...","[sekmaos, perintah, revisi, atur, laku, batas,...","[sekmaos, perintah, revisi, atur, laku, batas,..."
3,twitter,07/15/2021,8:12:00,RT @Daeng_Info:Rekaman CCTV Oknum Satpol PP di...,Negatif,RT Info Rekaman CCTV Oknum Satpol P di Kab Gow...,rt info rekaman cctv oknum satpol p di kab gow...,"[rt, info, rekaman, cctv, oknum, satpol, p, di...","[rt, info, rekaman, cctv, oknum, satpol, p, di...","[rt, info, rekam, cctv, oknum, satpol, p, di, ...","[info, rekam, cctv, oknum, satpol, p, kab, gow..."
4,twitter,07/19/2021,5:40:00,RT @detikcom:Kemenag mengumumkan petunjuk tekn...,Netral,RT Kemenag mengumumkan petunjuk teknis pelaksa...,rt kemenag mengumumkan petunjuk teknis pelaksa...,"[rt, kemenag, mengumumkan, petunjuk, teknis, p...","[rt, kemenag, mengumumkan, petunjuk, teknis, p...","[rt, kemenag, umum, tunjuk, teknis, laksana, k...","[kemenag, teknis, laksana, kurban, wilayah, pp..."


In [ ]:
#data_pakai.to_csv('data_baru+NO_Stopword.csv', index = False)